## Amazon DynamoDB Immersion Day - Part 2

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

dynamodb_client = boto3.client('dynamodb')

#### Product Catalog Table

In [3]:
product_table = DynamoTable()
product_table.select_table('ProductCatalog')
print(product_table)

Table info:
 - Table name: ProductCatalog
 - Table arn: arn:aws:dynamodb:us-east-1:006662441764:table/ProductCatalog
 - Table creation: 2024-08-28T09:27:41
 - Key schema: [{'AttributeName': 'Id', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'Id', 'AttributeType': 'N'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop', 'Value': 'ImmersionDay'}]



#### Forum Table

In [7]:
forum_table = DynamoTable()
forum_table.select_table('Forum')
print(forum_table)

Table info:
 - Table name: Forum
 - Table arn: arn:aws:dynamodb:us-east-1:006662441764:table/Forum
 - Table creation: 2024-08-28T09:28:05
 - Key schema: [{'AttributeName': 'Name', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'Name', 'AttributeType': 'S'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop', 'Value': 'ImmersionDay'}]



#### Thread Table

In [5]:
thread_table = DynamoTable()
thread_table.select_table('Thread')
print(thread_table)

Table info:
 - Table name: Thread
 - Table arn: arn:aws:dynamodb:us-east-1:006662441764:table/Thread
 - Table creation: 2024-08-28T09:28:30
 - Key schema: [{'AttributeName': 'ForumName', 'KeyType': 'HASH'}, {'AttributeName': 'Subject', 'KeyType': 'RANGE'}]
 - Attribute definitions: [{'AttributeName': 'ForumName', 'AttributeType': 'S'}, {'AttributeName': 'Subject', 'AttributeType': 'S'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop', 'Value': 'ImmersionDay'}]



#### Reply Table

In [6]:
reply_table = DynamoTable()
reply_table.select_table('Reply')
print(reply_table)

Table info:
 - Table name: Reply
 - Table arn: arn:aws:dynamodb:us-east-1:006662441764:table/Reply
 - Table creation: 2024-08-28T09:28:54
 - Key schema: [{'AttributeName': 'Id', 'KeyType': 'HASH'}, {'AttributeName': 'ReplyDateTime', 'KeyType': 'RANGE'}]
 - Attribute definitions: [{'AttributeName': 'Id', 'AttributeType': 'S'}, {'AttributeName': 'ReplyDateTime', 'AttributeType': 'S'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop', 'Value': 'ImmersionDay'}]



### Global Secondary Indexes

Hasta ahora nos habíamos ocupado de acceder a los datos basándonos en los atributos clave. Si queríamos buscar elementos basados en atributos no clave, teníamos que realizar una exploración completa de la tabla y utilizar condiciones de filtrado para encontrar lo que queríamos, lo que sería muy lento y costoso para los sistemas que funcionan a gran escala.

DynamoDB ofrece una función llamada Índices Secundarios Globales (GSI) que pivota automáticamente los datos en torno a diferentes claves de partición y ordenación. Los datos se pueden reagrupar y reordenar para permitir que se sirvan más patrones de acceso rápidamente con las API de consulta y exploración.

Recuerde el ejemplo anterior en el que queríamos encontrar todas las respuestas en la tabla Respuestas que fueron publicadas por el Usuario A:

In [13]:
reply_table.scan_att(att_name="PostedBy", query="User A", consumed_capacity=True)

Consumed Capacity: 0.5


[{'ReplyDateTime': '2015-09-15T19:58:22.947Z',
  'Message': 'DynamoDB Thread 1 Reply 1 text',
  'PostedBy': 'User A',
  'Id': 'Amazon DynamoDB#DynamoDB Thread 1'},
 {'ReplyDateTime': '2015-09-29T19:58:22.947Z',
  'Message': 'DynamoDB Thread 2 Reply 1 text',
  'PostedBy': 'User A',
  'Id': 'Amazon DynamoDB#DynamoDB Thread 2'},
 {'ReplyDateTime': '2015-10-05T19:58:22.947Z',
  'Message': 'DynamoDB Thread 2 Reply 2 text',
  'PostedBy': 'User A',
  'Id': 'Amazon DynamoDB#DynamoDB Thread 2'}]

Al ejecutar esa operación de escaneo, pudimos ver que el recuento devuelto era diferente del recuento escaneado. Si hubiera habido mil millones de elementos de respuesta pero sólo tres de ellos hubieran sido enviados por el usuario A, tendríamos que pagar (tanto en tiempo como en dinero) para escanear mil millones de elementos sólo para encontrar los tres que queríamos.

Con estos conocimientos sobre los GSI, podemos crear un GSI en la tabla Reply para dar servicio a este nuevo patrón de acceso. Los GSI pueden crearse y eliminarse en cualquier momento, incluso si la tabla ya contiene datos. Este nuevo GSI utilizará el atributo PostedBy como clave de partición (HASH) y mantendrá los mensajes ordenados por ReplyDateTime como clave de ordenación (RANGE). Queremos que todos los atributos de la tabla se copien (proyecten) en el GSI, por lo que utilizaremos el ProjectionType ALL. Observe que el nombre del índice que creamos es PostedBy-ReplyDateTime-gsi.


In [14]:
help(reply_table.create_global_secondary_index)

Help on method create_global_secondary_index in module spdynamodb.main:

create_global_secondary_index(att_name, att_type, index_name=None, sort_index=None, sort_type=None, proj_type='ALL', read_capacity=5, write_capacity=5) method of spdynamodb.main.DynamoTable instance
    Add a global secondary index to a DynamoDB table
    :param att_name: Name of attribute.
    :param att_type: Attribute type (S-String, N-Number, B-Binary).
    :param sort_index: Name of sort index. Default: None
    :param sort_type: Attribute type (S-String, N-Number, B-Binary). Default: None
    :param index_name: Name of index. Default: <att_name>-index
    :param proj_type: Represents attributes that are copied (projected) from the table into the global secondary index
                      (ALL, KEYS_ONLY, [list of INCLUDE non-key attribute])
    :param read_capacity: Read capacity units. Default: 5
    :param write_capacity: Write capacity units. Default: 5



In [16]:
reply_table.create_global_secondary_index(
    att_name="PostedBy",
    att_type="S",
    sort_index="ReplyDateTime",
    sort_type="S",
    index_name="PostedBy-ReplyDateTime-gsi"
)

status = reply_table.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = reply_table.check_status_gsi()
        time.sleep(30)
    end = time.time()
    minute = (end - start) / 60
    print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 6.03 minute


In [17]:
help(reply_table.query)

Help on method query in module spdynamodb.main:

query(pk_value, sk_value=None, index_name=None, consistent_read=False, consumed_capacity=None, limit=None, reverse=True, to_pandas=False) method of spdynamodb.main.DynamoTable instance
    Queries an Amazon DynamoDB table and returns the matching items.
    :param pk_value: Primary key value.
    :param sk_value: Sort key value if exist. Default: None.
    :param index_name: The name of the index to query. If None, then the table itself is queried.
    :param consistent_read: If True, then a strongly consistent read is used.
    :param consumed_capacity: Return the consumed capacity. Valid values: None, "TOTAL", "INDEXES". Default: None.
    :param to_pandas: If True, returns a pandas DataFrame. Default: True.
    :param limit: The maximum number of items to return. Default: None.
    :param reverse: If True, then the order of the search is ascending. If False, then the order of the search is descending. Default: True.
    :return: The i

In [19]:
reply_table.query(
    index_name="PostedBy-ReplyDateTime-gsi",
    pk_value="User A",
    consumed_capacity=True,
    to_pandas=True
)   

Consumed Capacity: 0.5


,ReplyDateTime,PostedBy,Message,Id
0,2015-09-15T19:58:22.947Z,User A,DynamoDB Thread 1 Reply 1 text,Amazon DynamoDB#DynamoDB Thread 1
1,2015-09-29T19:58:22.947Z,User A,DynamoDB Thread 2 Reply 1 text,Amazon DynamoDB#DynamoDB Thread 2
2,2015-10-05T19:58:22.947Z,User A,DynamoDB Thread 2 Reply 2 text,Amazon DynamoDB#DynamoDB Thread 2
